<a href="https://colab.research.google.com/github/isacariman/EnvironmentalAnalysis_Python/blob/main/C%C3%B3pia_de_queimadas_NorteBrasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd

In [67]:
queimadas = pd.read_csv("/content/queimadas_NBr.csv")
queimadas

,date,class,focuses,SIGLA_UF
0,2023/01,Desmatamento Consolidado,2,AC
1,2023/01,Desmatamento Consolidado,1,AP
2,2023/01,Desmatamento Consolidado,9,AM
3,2023/01,Desmatamento Consolidado,64,MA
4,2023/01,Desmatamento Consolidado,94,MT
...,...,...,...,...
1936,2019/12,Vegetacao Secundaria,159,MA
1937,2019/12,Vegetacao Secundaria,10,MT
1938,2019/12,Vegetacao Secundaria,253,PA
1939,2019/12,Vegetacao Secundaria,2,RO


In [68]:
queimadas.dtypes

date        object
class       object
focuses      int64
SIGLA_UF    object
dtype: object

In [69]:
queimadas['date'] = pd.to_datetime(queimadas['date'], format='%Y-%m-%d')
queimadas.dtypes
queimadas

,date,class,focuses,SIGLA_UF
0,2023-01-01,Desmatamento Consolidado,2,AC
1,2023-01-01,Desmatamento Consolidado,1,AP
2,2023-01-01,Desmatamento Consolidado,9,AM
3,2023-01-01,Desmatamento Consolidado,64,MA
4,2023-01-01,Desmatamento Consolidado,94,MT
...,...,...,...,...
1936,2019-12-01,Vegetacao Secundaria,159,MA
1937,2019-12-01,Vegetacao Secundaria,10,MT
1938,2019-12-01,Vegetacao Secundaria,253,PA
1939,2019-12-01,Vegetacao Secundaria,2,RO


In [70]:
queimadas['date'] = pd.to_datetime(queimadas['date'])
# Criando uma nova coluna 'ano' com apenas o ano
queimadas['ano'] = queimadas['date'].dt.year
queimadas

,date,class,focuses,SIGLA_UF,ano
0,2023-01-01,Desmatamento Consolidado,2,AC,2023
1,2023-01-01,Desmatamento Consolidado,1,AP,2023
2,2023-01-01,Desmatamento Consolidado,9,AM,2023
3,2023-01-01,Desmatamento Consolidado,64,MA,2023
4,2023-01-01,Desmatamento Consolidado,94,MT,2023
...,...,...,...,...,...
1936,2019-12-01,Vegetacao Secundaria,159,MA,2019
1937,2019-12-01,Vegetacao Secundaria,10,MT,2019
1938,2019-12-01,Vegetacao Secundaria,253,PA,2019
1939,2019-12-01,Vegetacao Secundaria,2,RO,2019


In [71]:
# Para poder montar nosso gráfico Choropleth, temos que baixar um geojson, ele carrega informações para desenhar as fronteiras políticas dos estados.
import requests
import json
# Requisição para baixar o geojson
map_json =  requests.get('https://github.com/datalivre/Conjunto-de-Dados/raw/master/br_states.json')
# Codificando a requisição em um json válido no python
geo_json_data = json.loads(map_json.text)

In [72]:
import folium
# Criamos um novo mapa
mapa = folium.Map(
    location=[-15.77972, -47.92972],
    zoom_start=3
)
# Adicionamos as informações do geojson ao mapa criado
folium.GeoJson(geo_json_data).add_to(mapa)
mapa

ValueError: Custom tiles must have an attribution.

In [53]:
mapa = folium.Map(
    location=[-15.77972, -47.92972],
    zoom_start=3)

# Objeto Choropleth
folium.Choropleth(

    # informações geográficas do Brasil no geojson
    geo_data = geo_json_data,
    name = "Exemplo choropleth",

    # o dataframe que vamos processar os dados
    data = queimadas,
    # detalhar as colunas do dataset
    columns = ["SIGLA_UF","focuses"],

    # YlGn é uma escala de amarelo (menor) para verde (maior)
    fill_color = "YlGn",
    # opacidade da escala do fundo do Choropleth
    fill_opacity = 0.7,
    # opacidade das linhas de divisão politica do Choropleth
    line_opacity = .1,
    # onde o Choropleth vai integrar geojson e o dataset df_estados
      key_on = "feature.id",
    #  Nome do mapa
    legend_name = "Número de focus de queimadas no Brasil",
).add_to(mapa)

mapa

ValueError: Custom tiles must have an attribution.

In [ ]:
!pip install folium --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
from folium import plugins
from branca.element import Figure
import numpy as np
import pandas as pd

# Your existing code for the map without custom tiles
# Substitua a linha existente onde você cria o objeto folium.Map por esta linha
mapa = folium.Map(location=[-15.77972, -47.92972], zoom_start=3, tiles='https://api.mapbox.com/styles/v1/mapbox/bright-v10/tiles/{z}/{x}/{y}?access_token=YOUR_MAPBOX_ACCESS_TOKEN', attr='Mapbox Attribution')

folium.Choropleth(
    geo_data=geo_json_data,
    name="Exemplo choropleth",
    data=queimadas,
    columns=["SIGLA_UF", "focuses"],
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.1,
    key_on="feature.id",
    legend_name="Número de focus de queimadas no Brasil",
).add_to(mapa)

# Your existing code for the slider
data_slider = []

for ano in queimadas.ano.unique():
    queimadas_ano = queimadas[queimadas["ano"] == ano].copy()
    queimadas_ano['focuses'] = pd.to_numeric(queimadas_ano['focuses'], errors='coerce')
    queimadas_ano['focuses'].replace([np.inf, -np.inf], np.nan, inplace=True)

    data_one_year = dict(
        type='choropleth',
        geojson=geo_json_data,
        locations=queimadas_ano['SIGLA_UF'],
        z=queimadas_ano['focuses'],
        colorscale='YlOrRd',
        colorbar=dict(
            len=0.7,
            ticklen=0.5,
            tickformat=".1f",
            thickness=30
        ),
        showscale=False,
    )

    data_slider.append(data_one_year)

# Create a slider control
slider_control = plugins.TimestampedGeoJson(
    { 'type': 'FeatureCollection', 'features': [] },
    period='P1Y',
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
).add_to(mapa)   # Add this line to add the slider to the map


# Add slider control to the map
slider_control.add_to(mapa)

# Save the map
mapa.save('map_with_slider.html')

# Display the map with slider
mapa
